In [ ]:
!pip install simpletransformers

     |████████████████████████████████| 225kB 3.1MB/s 
     |████████████████████████████████| 81kB 4.0MB/s 
     |████████████████████████████████| 8.2MB 16.0MB/s 
     |████████████████████████████████| 1.8MB 50.8MB/s 
     |████████████████████████████████| 245kB 51.7MB/s 
     |████████████████████████████████| 51kB 7.3MB/s 
     |████████████████████████████████| 122kB 56.7MB/s 
     |████████████████████████████████| 1.2MB 50.2MB/s 
     |████████████████████████████████| 2.3MB 50.5MB/s 
     |████████████████████████████████| 3.3MB 50.5MB/s 
     |████████████████████████████████| 81kB 11.7MB/s 
     |████████████████████████████████| 174kB 55.7MB/s 
     |████████████████████████████████| 112kB 60.9MB/s 
     |████████████████████████████████| 4.2MB 44.0MB/s 
     |████████████████████████████████| 102kB 13.3MB/s 
     |████████████████████████████████| 133kB 56.6MB/s 
     |████████████████████████████████| 245kB 44.1MB/s 
     |████████████████████████████████| 122kB 51.7MB/s

In [ ]:
import pandas as pd
import numpy as np
data = pd.read_csv("/content/drive/MyDrive/BIO_taggingdata_B_MUST.csv",encoding="latin1" )

In [ ]:
data.head(30)

,Sentence #,Word,Tag
0,Sentence :7,i,O
1,Sentence :7,am,O
2,Sentence :7,a,O
3,Sentence :7,19,O
4,Sentence :7,year,O
5,Sentence :7,old,O
6,Sentence :7,entrepreneur,O
7,Sentence :7,and,O
8,Sentence :7,manager,O
9,Sentence :7,of,O


In [ ]:
data=data.replace(r'^\s*$', np.nan, regex=True)

In [ ]:
# data=data.drop(columns=["Unnamed: 0"])

In [ ]:
data =data.fillna(method ="ffill")
data.head(15)

,Sentence #,Word,Tag
0,Sentence :7,i,O
1,Sentence :7,am,O
2,Sentence :7,a,O
3,Sentence :7,19,O
4,Sentence :7,year,O
5,Sentence :7,old,O
6,Sentence :7,entrepreneur,O
7,Sentence :7,and,O
8,Sentence :7,manager,O
9,Sentence :7,of,O


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [ ]:
data["Sentence #"] = LabelEncoder().fit_transform(data["Sentence #"] )
data.head()

,Sentence #,Word,Tag
0,232,i,O
1,232,am,O
2,232,a,O
3,232,19,O
4,232,year,O


In [ ]:
data['Tag'].value_counts()

O        110225
B-ORG      1971
I-ORG       148
Name: Tag, dtype: int64

In [ ]:
data.loc[(data['Tag'] == 'B'), 'Tag'] = 'B-ORG'
data.loc[(data['Tag'] == 'I'), 'Tag'] = 'I-ORG'

In [ ]:
data.rename(columns={"Sentence #":"sentence_id","Word":"words","Tag":"labels"}, inplace =True)

In [ ]:
data.labels.unique()

array(['O', 'B-ORG', 'I-ORG'], dtype=object)

In [ ]:
data.labels.value_counts()

O        110225
B-ORG      1971
I-ORG       148
Name: labels, dtype: int64

In [ ]:
data["labels"] = data["labels"].str.upper()

In [ ]:
X= data[["sentence_id","words"]]
Y =data["labels"]

In [ ]:
X

,sentence_id,words
0,232,i
1,232,am
2,232,a
3,232,19
4,232,year
...,...,...
112339,343,higher
112340,343,education
112341,343,at
112342,343,xerox


In [ ]:
Y.unique()

array(['O', 'B-ORG', 'I-ORG'], dtype=object)

In [ ]:
Y.value_counts()

O        110225
B-ORG      1971
I-ORG       148
Name: labels, dtype: int64

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size =0.2)

In [ ]:
#building up train data and test data
train_data = pd.DataFrame({"sentence_id":x_train["sentence_id"],"words":x_train["words"],"labels":y_train})
test_data = pd.DataFrame({"sentence_id":x_test["sentence_id"],"words":x_test["words"],"labels":y_test})

In [ ]:
train_data['labels'].value_counts()

O        88167
B-ORG     1592
I-ORG      116
Name: labels, dtype: int64

#Model Training

In [ ]:
from simpletransformers.ner import NERModel,NERArgs

In [ ]:
label = data["labels"].unique().tolist()
label

['O', 'B-ORG', 'I-ORG']

In [ ]:
args = NERArgs()
args.num_train_epochs =3
args.learning_rate = 1e-4
args.overwrite_output_dir =True
args.train_batch_size = 32
args.eval_batch_size = 32

In [ ]:
model = NERModel('bert', 'bert-base-uncased',labels=label,args =args)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

In [ ]:
result, model_outputs, preds_list = model.eval_model(test_data)

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/11 [00:00<?, ?it/s]

In [ ]:
result #without fine tuning

{'eval_loss': 0.9990108446641401,
 'f1_score': 0.032120229851186094,
 'precision': 0.01708196207490989,
 'recall': 0.2684729064039409}

In [ ]:
# X=np.array(preds_list)
# X.shape

In [ ]:
k=[]
for i in preds_list:
  for j in i:
    k.append(j)
len(k)

22223

In [ ]:
train_data['sentence_id'].nunique()

344

In [ ]:
groups = train_data.groupby('sentence_id').size()
groups

sentence_id
0      420
1      330
2      294
3      222
4      246
      ... 
339    360
340    175
341    165
342    274
343    227
Length: 344, dtype: int64

In [ ]:
test_data['sentence_id'].nunique()

344

In [ ]:
all_list = test_data['sentence_id'].unique()
groups = test_data.groupby('sentence_id').size()

In [ ]:
# groups

In [ ]:
# l1 = groups

In [ ]:
# l2=dataff.count(axis=1)
# type(l2)

In [ ]:
# dataff = pd.DataFrame(preds_list)
# dataff.head()

In [ ]:
# dataff.count(axis=1) #Counting no. of words predicted in a sentence

In [ ]:
# dict = {'test_data': l1, 'pred_length': l2}
# tally_df = pd.DataFrame(dict)
# tally_df 

In [ ]:
# tally_df

In [ ]:
all_list = test_data['sentence_id'].unique()
groups = test_data.groupby('sentence_id')

In [ ]:
comp_acc = []
for i in range(0,len(tally_df)):
  temp = groups.get_group(i)
  label_list = temp['labels'].tolist()
  if tally_df.iloc[i][0]!=tally_df.iloc[i][1]:
    comp_acc=comp_acc+ label_list[:tally_df.iloc[i][1]]
    # print(tally_df.iloc[i][0]," ",tally_df.iloc[i][1])
  else:
    comp_acc=comp_acc+ label_list

In [ ]:
len(comp_acc)

22223

In [ ]:
# from google.colab import files
# tally_df.to_csv('length_check.csv') 
# files.download('length_check.csv')

In [ ]:
from sklearn import metrics
# Predicted values
y_pred=k
# Actual values
y_true = comp_acc
print("CONFUSION MATRIX")
print(metrics.confusion_matrix(y_true, y_pred, labels=["B-ORG", "I-ORG","O"]),"\n")
print(metrics.classification_report(y_true, y_pred, labels=["B-ORG", "I-ORG", "O"]))

CONFUSION MATRIX
[[   88    95   193]
 [    3    14    15]
 [ 3928  4431 13456]] 

              precision    recall  f1-score   support

       B-ORG       0.02      0.23      0.04       376
       I-ORG       0.00      0.44      0.01        32
           O       0.98      0.62      0.76     21815

    accuracy                           0.61     22223
   macro avg       0.34      0.43      0.27     22223
weighted avg       0.97      0.61      0.75     22223



In [ ]:
k.count("B-ORG")

4019

B- ORG

In [ ]:
model.train_model(train_data,eval_data = test_data,acc=accuracy_score)

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/11 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/11 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/11 [00:00<?, ?it/s]

(33, 0.038730635392394935)

In [ ]:
# len(y_test), len(k)

(22469, 22223)

In [ ]:
result, model_outputs, preds_list = model.eval_model(test_data)
result #after fine tuning

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/11 [00:00<?, ?it/s]

{'eval_loss': 0.03806282122704116,
 'f1_score': 0.689655172413793,
 'precision': 0.7836990595611285,
 'recall': 0.6157635467980296}

In [ ]:
k=[]
for i in preds_list:
  for j in i:
    k.append(j)
len(k)

22223

In [ ]:
from sklearn import metrics
# Predicted values
y_pred=k
# Actual values
y_true = comp_acc
print("CONFUSION MATRIX")
print(metrics.confusion_matrix(y_true, y_pred, labels=["B-ORG", "I-ORG","O"]),"\n")
print(metrics.classification_report(y_true, y_pred, labels=["B-ORG", "I-ORG", "O"]))

CONFUSION MATRIX
[[  251     0   125]
 [    1     0    31]
 [   67     0 21748]] 

              precision    recall  f1-score   support

       B-ORG       0.79      0.67      0.72       376
       I-ORG       0.00      0.00      0.00        32
           O       0.99      1.00      0.99     21815

    accuracy                           0.99     22223
   macro avg       0.59      0.55      0.57     22223
weighted avg       0.99      0.99      0.99     22223



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
prediction, model_output = model.predict(["Mohit works at microsoft and is expert in python and mysql. He uses slack for communication"])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
y_pred

[['O',
  'B-ORG',
  'B-ORG',
  'O',
  'O',
  'B-ORG',
  'O',
  'O',
  'B-ORG',
  'O',
  'O',
  'B-ORG',
  'B-ORG',
  'B-ORG',
  'B-ORG',
  'B-ORG',
  'O',
  'O',
  'O',
  'O',
  'B-ORG',
  'O',
  'B-ORG',
  'O',
  'B-ORG',
  'B-ORG',
  'B-ORG',
  'B-ORG',
  'B-ORG',
  'B-ORG',
  'O',
  'B-ORG',
  'B-ORG',
  'I-ORG',
  'B-ORG',
  'B-ORG',
  'B-ORG',
  'B-ORG',
  'B-ORG',
  'B-ORG',
  'B-ORG',
  'B-ORG',
  'B-ORG',
  'B-ORG',
  'O',
  'B-ORG',
  'B-ORG',
  'B-ORG',
  'O',
  'B-ORG',
  'B-ORG',
  'B-ORG',
  'O',
  'O',
  'B-ORG',
  'O',
  'B-ORG',
  'O',
  'O',
  'O',
  'O',
  'B-ORG',
  'B-ORG',
  'O',
  'B-ORG',
  'B-ORG',
  'B-ORG',
  'B-ORG',
  'B-ORG',
  'B-ORG',
  'O',
  'B-ORG',
  'O',
  'B-ORG',
  'B-ORG',
  'O',
  'B-ORG',
  'O',
  'B-ORG',
  'O',
  'B-ORG',
  'B-ORG',
  'I-ORG',
  'B-ORG',
  'O',
  'B-ORG',
  'B-ORG',
  'O',
  'B-ORG',
  'O',
  'B-ORG',
  'B-ORG',
  'B-ORG',
  'O',
  'B-ORG',
  'B-ORG',
  'O',
  'O',
  'O',
  'B-ORG',
  'B-ORG',
  'B-ORG',
  'O',
  'B-ORG',
  'B

In [ ]:
prediction

[[{'Mohit': 'O'},
  {'works': 'O'},
  {'at': 'O'},
  {'microsoft': 'O'},
  {'and': 'O'},
  {'is': 'O'},
  {'expert': 'O'},
  {'in': 'O'},
  {'python': 'O'},
  {'and': 'O'},
  {'mysql.': 'O'},
  {'He': 'O'},
  {'uses': 'O'},
  {'slack': 'O'},
  {'for': 'O'},
  {'communication': 'O'}]]

In [ ]:
model_output

[[{'Mohit': [[3.504, -1.602, -2.031]]},
  {'works': [[2.78, -1.162, -1.835]]},
  {'at': [[3.006, -1.326, -2.006]]},
  {'microsoft': [[-0.4763, 1.0205, -0.4724]]},
  {'and': [[3.09, -1.47, -2.035]]},
  {'is': [[3.074, -1.528, -1.919]]},
  {'expert': [[2.998, -1.412, -1.938]]},
  {'in': [[3.055, -1.326, -2.041]]},
  {'python': [[0.2766, 0.5596, -0.958]]},
  {'and': [[3.023, -1.3545, -2.135]]},
  {'mysql.': [[1.903, -0.685, -1.345],
    [1.962, -0.526, -1.3545],
    [2.197, -0.5596, -1.774],
    [1.118, -0.2651, -1.229],
    [2.94, -1.432, -1.883]]},
  {'He': [[3.467, -1.67, -2.045]]},
  {'uses': [[3.428, -1.52, -1.955]]},
  {'slack': [[2.273, -1.025, -1.596]]},
  {'for': [[3.408, -1.279, -2.01]]},
  {'communication': [[1.564, -0.5176, -1.558]]}]]

In [ ]:
import nltk
import re
from bs4 import BeautifulSoup

nltk.download("punkt")

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    return text



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
